In [12]:
import pandas as pd
import os

file_names = os.listdir('../AnxietyMeditations/')
# Create Dictionary for File Name and Text
file_name_and_text = {}
for file in file_names:
    f = open('../AnxietyMeditations/' + file, "r",
             encoding='unicode_escape')
    file_name_and_text[file] = f.read()
file_data = (pd.DataFrame.from_dict(file_name_and_text, orient='index')
             .reset_index().rename(index=str, columns={'index': 'file_name', 0: 'text'}))

file_data

,file_name,text
0,mm42.txt,hello welcome to today's gentle movement pract...
1,mm56.txt,take a load off right now set yourself down an...
2,mm81.txt,holding on to negative thoughts and emotions c...
3,mm95.txt,Find a comfortable position sitting or lying d...
4,mm2.txt,welcome to this guided meditation but\nwas thi...
...,...,...
96,mm72.txt,- Hi everyone. Welcome to Yoga With Adriene.\n...
97,mm99.txt,Our own self-talk can contribute to the pain a...
98,mm98.txt,Take a moment to settle into a comfortable pos...
99,mm73.txt,"20 Minute Guided Meditation for Depression, An..."


In [16]:

import math
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

In [30]:
from nltk.corpus import stopwords
en_stop_words = set(stopwords.words('english'))
list(en_stop_words)[:10]

def clean_tweets(df=file_data, 
                 text_col='text', 
                ):
    
    df_copy = df.copy()
    
    # drop rows with empty values
    df_copy.dropna(inplace=True)
    
    # format the date
    #df_copy[date_col] = df_copy[date_col].apply(lambda row: datetime.strptime(row, '%m-%d-%Y %H:%M:%S'))
    
    # filter rows older than a given date
    #df_copy = df_copy[df_copy[date_col] >=start_datetime]
    
    # lower the tweets
    df_copy['preprocessed_' + text_col] = df_copy[text_col].str.lower()
    
    # filter out stop words and URLs
    en_stop_words = set(stopwords.words('english'))
    en_stop_words.add('music')
    extended_stop_words = en_stop_words | \
                        {
                            '&amp;', 'rt',                           
                            'th','co', 're', 've', 'kim', 'daca'
                        }
    url_re = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'        
    df_copy['preprocessed_' + text_col] = df_copy['preprocessed_' + text_col].apply(lambda row: ' '.join(
        [word for word in row.split() if (not word in en_stop_words) and (not re.match(url_re, word))]))
    
    # tokenize the tweets
    tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
    df_copy['tokenized_' + text_col] = df_copy['preprocessed_' + text_col].apply(lambda row: tokenizer.tokenize(row))
    
    return df_copy
  
#df_tweets = pd.read_csv('trump_tweets.csv')
df_clean = clean_tweets(file_data)
df_clean.head()

,file_name,text,preprocessed_text,tokenized_text
0,mm42.txt,hello welcome to today's gentle movement pract...,hello welcome today's gentle movement practice...,"[hello, welcome, today's, gentle, movement, pr..."
1,mm56.txt,take a load off right now set yourself down an...,take load right set relax best moment fully ap...,"[take, load, right, set, relax, best, moment, ..."
2,mm81.txt,holding on to negative thoughts and emotions c...,holding negative thoughts emotions heavy creat...,"[holding, negative, thoughts, emotions, heavy,..."
3,mm95.txt,Find a comfortable position sitting or lying d...,find comfortable position sitting lying down. ...,"[find, comfortable, position, sitting, lying, ..."
4,mm2.txt,welcome to this guided meditation but\nwas thi...,welcome guided meditation thing present practi...,"[welcome, guided, meditation, thing, present, ..."


In [31]:
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(str)
    bag_of_words = vect.transform(str)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
get_most_freq_words([ word for line in df_clean.tokenized_text for word in line],10)

[('body', 912),
 ('music', 911),
 ('breath', 803),
 ('let', 712),
 ('feel', 666),
 ('go', 643),
 ('allow', 426),
 ('take', 416),
 ('mind', 400),
 ('moment', 396)]

In [35]:
import matplotlib.pyplot as plt
# build a dictionary where for each tweet, each word has its own id.
# We have 6882 tweets and 10893 words in the dictionary.
tweets_dictionary = Dictionary(df_clean.tokenized_text)

# build the corpus i.e. vectors with the number of occurence of each word per tweet
tweets_corpus = [tweets_dictionary.doc2bow(
    tweet) for tweet in df_clean.tokenized_text]

# compute coherence
tweets_coherence = []
for nb_topics in range(1, 36):
    lda = LdaModel(tweets_corpus, num_topics=nb_topics,
                   id2word=tweets_dictionary, passes=10)
    cohm = CoherenceModel(model=lda, corpus=tweets_corpus,
                          dictionary=tweets_dictionary, coherence='u_mass')
    coh = cohm.get_coherence()
    tweets_coherence.append(coh)

# visualize coherence
plt.figure(figsize=(10, 5))
plt.plot(range(1, 36), tweets_coherence)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
